---
title: Neuroglancer JupyterLab Binder demo
---


A brief demonstration of the [neuroglancer package repurposed for JupyterLab](https://github.com/2i2c-org/jupyter-neuroglancer). You can run these cells from top-to-bottom to see how Neuroglancer can be used in this interface.

:::{note}
This notebook was adapted from the [neuroglancer jupyter example notebook](https://github.com/google/neuroglancer/blob/master/python/examples/jupyter-notebook-demo.ipynb), with a two line addition to enable [jupyter-neuroglancer](https://github.com/2i2c-org/jupyter-neuroglancer) and some formatting changes for readability with [MyST-JupyterLab engine](https://mystmd.org).
:::

In [1]:
import neuroglancer
import numpy as np

## Create a new viewer

Create a new (initially empty) viewer.  This starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [2]:
viewer = neuroglancer.Viewer()

Got my-action
  Mouse position: [2977.1106 2984.811  3000.5002]
  Layer selected values: Map({"image": {"value": 104}, "overlay": {}})
Got my-action
  Mouse position: [2977.1106 2984.811  3000.5002]
  Layer selected values: Map({"image": {"value": 104}, "overlay": {}})


## Open the viewer in JupyterLab
Open the viewer as a pane in JupyterLab, using [jupyter-neuroglancer](https://github.com/yuvipanda/jupyter-neuroglancer). This handles securely authenticating when running remotely in a JupyterHub if necessary

In [3]:
from jupyter_neuroglancer import display_in_sidecar
display_in_sidecar(viewer)

## Add some image layers from cell data
Add some example layers using the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset).

In [5]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/image')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth', selected_alpha=0.3)


## Highlight a subset of the data
Display a numpy array as an additional layer.  A reference to the numpy array is kept only as long as the layer remains in the viewer.

Move the viewer position.

In [9]:
with viewer.txn() as s:
    s.voxel_coordinates = [3000.5, 3000.5, 3000.5]

Hide the segmentation layer.

In [7]:
with viewer.txn() as s:
    s.layers['segmentation'].visible = False

In [8]:
import tensorstore as ts

image_vol = await ts.open({'driver': 'neuroglancer_precomputed', 'kvstore': 'gs://neuroglancer-public-data/flyem_fib-25/image/'})
a = np.zeros((200,200,200), np.uint8)
def make_thresholded(threshold):
  a[...] = image_vol[3000:3200,3000:3200,3000:3200][...,0].read().result() > threshold
make_thresholded(110)
# This volume handle can be used to notify the viewer that the data has changed.
volume = neuroglancer.LocalVolume(
    a,
    dimensions=neuroglancer.CoordinateSpace(
        names=['x', 'y', 'z'],
        units='nm',
        scales=[8, 8, 8],
    ),
    voxel_offset=[3000, 3000, 3000])
with viewer.txn() as s:
  s.layers['overlay'] = neuroglancer.ImageLayer(
        source=volume,
      # Define a custom shader to display this mask array as red+alpha.
        shader="""
void main() {
  float v = toNormalized(getDataValue(0)) * 255.0;
  emitRGBA(vec4(v, 0.0, 0.0, v));
}
""",
    )

I0000 00:00:1720026408.967548 21520099 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
W0000 00:00:1720026408.991029 21524268 curl_transport.cc:452] Error [6]=Couldn't resolve host name in curl operation
Could not resolve host: metadata.google.internal
E0000 00:00:1720026408.991158 21524273 google_auth_provider.cc:187] Could not find the credentials file in the standard gcloud location [/Users/choldgraf/.config/gcloud/application_default_credentials.json]. You may specify a credentials file using $GOOGLE_APPLICATION_CREDENTIALS, or to use Google application default credentials, run: gcloud auth application-default login


Modify the overlay volume, and call `invalidate()` to notify the Neuroglancer client.

In [10]:
make_thresholded(100)
volume.invalidate()

## See the current state of the viewer

Print the neuroglancer viewer state.  The Neuroglancer Python library provides a set of Python objects that wrap the JSON-encoded viewer state.  `viewer.state` returns a read-only snapshot of the state.  To modify the state, use the `viewer.txn()` function, or `viewer.set_state`.

In [11]:
viewer.state

ViewerState({"dimensions": {"x": [8e-09, "m"], "y": [8e-09, "m"], "z": [8e-09, "m"]}, "position": [3000.5, 3000.5, 3000.5], "crossSectionScale": 1, "projectionScale": 8192, "layers": [{"type": "image", "source": "precomputed://gs://neuroglancer-public-data/flyem_fib-25/image", "tab": "source", "name": "image"}, {"type": "segmentation", "source": "precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth", "tab": "source", "selectedAlpha": 0.3, "segments": [], "name": "segmentation", "visible": false}, {"type": "image", "source": "python://volume/e98d9a6438faf27e8e5537469b8a57f92fde3363.231bf082be1b727fd32662478b2c661ebf2e15f1", "tab": "source", "shader": "\nvoid main() {\n  float v = toNormalized(getDataValue(0)) * 255.0;\n  emitRGBA(vec4(v, 0.0, 0.0, v));\n}\n", "name": "overlay"}], "layout": "4panel"})

Print the set of selected segments.|

In [ ]:
viewer.state.layers['segmentation'].segments

## Manually update the state of the viewer

Update the state by calling `set_state` directly.

In [12]:
import copy

new_state = copy.deepcopy(viewer.state)
new_state.layers['segmentation'].segments.add(10625)
viewer.set_state(new_state)

'c97d2ac4b58ee82735710da1ed575ccd12c07909'

## Create a keyboard shortcut action

Bind the 't' key in neuroglancer to a Python action.

In [14]:
num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
      st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
                                     (num_actions, s.mouse_voxel_coordinates))
    print('Got my-action')
    print(f'  Mouse position: {s.mouse_voxel_coordinates}')
    print(f'  Layer selected values: {s.selected_values}')
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyt'] = 'my-action'
    s.status_messages['hello'] = 'Welcome to this example'

Change the view layout to 3-d.

In [15]:
with viewer.txn() as s:
    s.layout = '3d'
    s.projection_scale = 3000

## Take a screenshot

Take a screenshot (useful for creating publication figures, or for generating videos).  While capturing the screenshot, we hide the UI and specify the viewer size so that we get a result independent of the browser size.

In [ ]:
from ipywidgets import Image

screenshot = viewer.screenshot(size=[1000, 1000])
screenshot_image = Image(value=screenshot.screenshot.image)
screenshot_image

Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['image', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['segmentation'])])

Remove the overlay layer.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['image']),
         neuroglancer.LayerGroupViewer(layers=['segmentation'])])

## Create a publicly-sharable URL

Create a publicly sharable URL to the viewer state (only works for external data sources, not layers served from Python).  The Python objects for representing the viewer state (`neuroglancer.ViewerState` and friends) can also be used independently from the interactive Python-tied viewer to create Neuroglancer links.

In [ ]:
print(neuroglancer.to_url(viewer.state))